In [5]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
GPU available ? :  True


In [6]:
import pandas as pd
import tqdm
import numpy as np
import pickle

In [7]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [9]:
data = pd.read_csv('../../Data/movie_201130_table_3_4.csv')

* column name str -> int

In [12]:
data.columns

Index(['sa_id', 'album_id', 'buy_history_price', 'buy_tot', 'release_date',
       'pr_info', 'run_time', 'movie_meta_price', 'genre_NA', 'genre_SF',
       'genre_가족', 'genre_공포', 'genre_극장판애니', 'genre_기타', 'genre_다큐멘터리',
       'genre_독립영화', 'genre_드라마', 'genre_로맨스', 'genre_멜로', 'genre_모험',
       'genre_범죄', 'genre_스릴러', 'genre_애니메이션', 'genre_애정', 'genre_액션',
       'genre_전쟁', 'genre_코미디', 'genre_판타지', 'genre_mid_NA', 'genre_mid_SF',
       'genre_mid_공포', 'genre_mid_극장판애니', 'genre_mid_기타', 'genre_mid_다큐멘터리',
       'genre_mid_드라마', 'genre_mid_로맨스', 'genre_mid_스릴러', 'genre_mid_시리즈',
       'genre_mid_액션', 'genre_mid_코미디', 'genre_mid_판타지', 'genre_small_NA',
       'genre_small_SF', 'genre_small_UHD', 'genre_small_가족',
       'genre_small_공연예술', 'genre_small_공포', 'genre_small_단편',
       'genre_small_독립', 'genre_small_드라마', 'genre_small_로맨스',
       'genre_small_모험', 'genre_small_무협', 'genre_small_성인', 'genre_small_스릴러',
       'genre_small_액션', 'genre_small_어린이', 'genre_small_에로',
 

In [13]:
final_genre = ['genre_NA', 'genre_SF',
       'genre_가족', 'genre_공포', 'genre_극장판애니', 'genre_기타', 'genre_다큐멘터리',
       'genre_독립영화', 'genre_드라마', 'genre_로맨스', 'genre_멜로', 'genre_모험',
       'genre_범죄', 'genre_스릴러', 'genre_애니메이션', 'genre_애정', 'genre_액션',
       'genre_전쟁', 'genre_코미디', 'genre_판타지', 'genre_mid_NA', 'genre_mid_SF',
       'genre_mid_공포', 'genre_mid_극장판애니', 'genre_mid_기타', 'genre_mid_다큐멘터리',
       'genre_mid_드라마', 'genre_mid_로맨스', 'genre_mid_스릴러', 'genre_mid_시리즈',
       'genre_mid_액션', 'genre_mid_코미디', 'genre_mid_판타지', 'genre_small_NA',
       'genre_small_SF', 'genre_small_UHD', 'genre_small_가족',
       'genre_small_공연예술', 'genre_small_공포', 'genre_small_단편',
       'genre_small_독립', 'genre_small_드라마', 'genre_small_로맨스',
       'genre_small_모험', 'genre_small_무협', 'genre_small_성인', 'genre_small_스릴러',
       'genre_small_액션', 'genre_small_어린이', 'genre_small_에로',
       'genre_small_예고편', 'genre_small_이벤트', 'genre_small_캐치온',
       'genre_small_코미디', 'genre_small_판타지']

In [14]:
re_dict = {}
for i in range(len(final_genre)):
    re_dict[final_genre[i]] = str(i)

In [16]:
data = data.rename(columns = re_dict)

In [17]:
data

,sa_id,album_id,buy_history_price,buy_tot,release_date,pr_info,run_time,movie_meta_price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,i30_meta_price
0,205080062020,M0119C2391PPV00,2500.0,0,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
1,205090440924,M0119C2391PPV00,2500.0,3,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
2,205091021367,M0119C2391PPV00,2500.0,1,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
3,205091176569,M0119C2391PPV00,2500.0,8,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
4,205124429679,M0119C2391PPV00,0.0,1,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14474135,500203264447,M0116C1107PPV00,1400.0,16,20161130,3,86,1400.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1400.0
14474136,500203997743,M01203N096PPV00,4900.0,0,20001215,1,84,4900.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4900.0
14474137,500204543205,M01151Q206PPV00,4500.0,0,20141224,5,119,6500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6500.0
14474138,500206206352,M0118AN125PPV00,0.0,0,20181011,4,111,1400.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1400.0


In [18]:
data.to_csv('../../Data/movie_201130_table_3_4_5.csv', index=None)

In [19]:
data

,sa_id,album_id,buy_history_price,buy_tot,release_date,pr_info,run_time,movie_meta_price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,i30_meta_price
0,205080062020,M0119C2391PPV00,2500.0,0,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
1,205090440924,M0119C2391PPV00,2500.0,3,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
2,205091021367,M0119C2391PPV00,2500.0,1,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
3,205091176569,M0119C2391PPV00,2500.0,8,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
4,205124429679,M0119C2391PPV00,0.0,1,20191107,4,106,2500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14474135,500203264447,M0116C1107PPV00,1400.0,16,20161130,3,86,1400.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1400.0
14474136,500203997743,M01203N096PPV00,4900.0,0,20001215,1,84,4900.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4900.0
14474137,500204543205,M01151Q206PPV00,4500.0,0,20141224,5,119,6500.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6500.0
14474138,500206206352,M0118AN125PPV00,0.0,0,20181011,4,111,1400.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1400.0
